In [2]:
from datetime import datetime
from dateutil.parser import parse
from langchain_core.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import yfinance as yf
import pandas as pd
import ta
from typing import List

In [3]:
# 讀取環境變數(OpenAI API Key)
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# 定義實際抓取股價的工具(使用yfinance)
@tool
def get_stock_price_history(symbol: str, days: int) -> str:
    """
    查詢股票代碼在 Yahoo Finance 上最近指定天數 (days) 的每日收盤價
    (symbol: 股票代碼，例如 'AAPL', '2330.TW'；days: 天數，例如 30, 90)
    """
    stock = yf.Ticker(symbol)
    try:
        df = stock.history(period=f"{days}d")
        if df.empty:
            return f"找不到 {symbol} 的歷史價格資料，請確認股票代碼是否正確"
        closes = df["Close"]
        result = "\n".join([f"{date.date()}: {price:.2f}" for date, price in closes.items()])
        return result
    except Exception as e:
        return f"查詢 {symbol} 最近 {days} 天價格資料時發生錯誤：{str(e)}"

In [5]:
# 計算簡單移動平均函數
@tool
def compute_sma_from_list(prices: List[float], window: int) -> List[float]:
    """
    根據輸入的股價價格列表 prices 與計算視窗 window 計算並回傳 SMA 列表。

    參數:
    - prices: List[float]，股價收盤價序列，例如 [123.5, 124.0, 125.2, ...]
    - window: int，SMA 計算視窗天數，例如 5, 10, 20

    回傳:
    - List[float]，計算後的 SMA 序列，若不足 window 則會顯示 NaN。
    """
    price_series = pd.Series(prices)
    sma_series = ta.trend.sma_indicator(price_series, window=window)
    # 將 NaN 轉為 None，以便於 JSON 可序列化
    return sma_series.round(2).tolist()

In [6]:
# 取得最新淨利
@tool
def get_latest_net_income(symbol: str) -> float:
    """
    查詢股票代碼在 Yahoo Finance 上的最新一期淨利 (Net Income)。

    參數:
    - symbol: 股票代碼，例如 'AAPL', 'TSLA', '2330.TW'。

    回傳:
    - 成功：最新一期淨利 (float)
    - 若找不到則回傳 -1
    """
    try:
        stock = yf.Ticker(symbol)
        is_ = stock.financials

        if is_.empty:
            return -1

        ni_candidates = [idx for idx in is_.index if 'net income' in idx.lower()]
        if not ni_candidates:
            return -1

        ni_label = ni_candidates[0]
        net_income = is_.loc[ni_label].iloc[0]

        return float(net_income)

    except Exception:
        return -1

In [7]:
# 取得最新股東權益
@tool
def get_latest_total_equity(symbol: str) -> float:
    """
    查詢股票代碼在 Yahoo Finance 上的最新一期總股東權益 (Total Stockholder Equity)。

    參數:
    - symbol: 股票代碼，例如 'AAPL', 'TSLA', '2330.TW'。

    回傳:
    - 成功：最新一期股東權益 (float)
    - 若找不到則回傳 -1
    """
    try:
        stock = yf.Ticker(symbol)
        bs = stock.balance_sheet

        if bs.empty:
            return -1

        equity_candidates = [idx for idx in bs.index if 'equity' in idx.lower()]
        if not equity_candidates:
            return -1

        equity_label = equity_candidates[0]
        total_equity = bs.loc[equity_label].iloc[0]

        return float(total_equity)

    except Exception:
        return -1

In [8]:
# 計算 ROE
@tool
def compute_roe(net_income: float, total_equity: float) -> str:
    """
    根據輸入的淨利與股東權益計算 ROE (Return on Equity)。

    參數:
    - net_income: float，最新一期淨利 (Net Income)，例如 1000000.0
    - total_equity: float，最新一期股東權益 (Total Stockholder Equity)，例如 5000000.0

    回傳:
    - 成功：回傳 "最新 ROE (股東權益報酬率) 約為 XX.XX%"
    - 失敗：回傳錯誤提示
    """
    try:
        if total_equity == 0:
            return "股東權益為 0，無法計算 ROE。"

        roe = net_income / total_equity * 100
        return f"最新 ROE (股東權益報酬率) 約為 {roe:.2f}%"
    
    except Exception as e:
        return f"計算 ROE 時發生錯誤：{str(e)}"

In [9]:
# 組成工具列表
tools = [
    get_stock_price_history, 
    compute_sma_from_list,
    get_latest_net_income,
    get_latest_total_equity,
    compute_roe
]

In [10]:
# 使用 gpt-3.5-turbo
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [11]:
# 使用 Function Calling 模式
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

C:\Users\User\AppData\Local\Temp\ipykernel_26508\2736381593.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
# 測試輸入
# query = "請問2330.TW的股價是多少?"
# query = "請問台積電的股價是多少?"
# query = "請問中華電的股價是多少?"
# query = "請問中華電最近30日的股價是多少?"
# query = "請問中華電5日均線是多少?"
# query = "請問中華電的淨利,股東權益與ROE是是多少?"
# query = "請問AAPL的淨利,股東權益與ROE是多少?"
# query = "請問中華電的ROE是多少?"
query = "請問中華電的ROE是多少?請給我你的思考流程"

# 執行呼叫
response = agent.invoke({"input": query})

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `get_latest_net_income` with `{'symbol': '2412.TW'}`


37189000000.0
Invoking: `get_latest_total_equity` with `{'symbol': '2412.TW'}`


382826000000.0
Invoking: `compute_roe` with `{'net_income': 37189000000, 'total_equity': 382826000000}`


最新 ROE (股東權益報酬率) 約為 9.71%中華電的最新 ROE (股東權益報酬率) 約為 9.71%。

我的思考流程如下：
1. 首先，我查詢了中華電的最新一期淨利為 37,189,000,000。
2. 接著，我查詢了中華電的最新一期總股東權益為 382,826,000,000。
3. 最後，我使用這兩個數據計算了中華電的 ROE，結果為約為 9.71%。

> Finished chain.
中華電的最新 ROE (股東權益報酬率) 約為 9.71%。

我的思考流程如下：
1. 首先，我查詢了中華電的最新一期淨利為 37,189,000,000。
2. 接著，我查詢了中華電的最新一期總股東權益為 382,826,000,000。
3. 最後，我使用這兩個數據計算了中華電的 ROE，結果為約為 9.71%。
